# Julia 檔案處理與資料庫連線

## 1. 流 (Stream) 與 IO

各種 Julia 不同 stream 的上層型別均為 IO，也都繼承了 read() 和 write() 函式。

最常見的標準流為輸入 (stdin)、輸出 (stdout)、錯誤 (stderr)。

### 1.1 標準輸入 (stdin)、輸出 (stdout)、錯誤 (stderr)

In [1]:
# 標準輸入 (stdin)
# 並且將輸入值指定給變數 a
a = readline()

stdin>  1


"1"

In [2]:
a

"1"

在 Jupyter / IJulia 環境中, 無法實現用 `read()` 函式來做為 stdin, IJulia 僅實作了 `readline()`

在 REPL 中執行標準輸入 `read()`, 例如:
```julia
# 標準輸入, 並指定輸入值為 Char
read(stdin, Char)
```

In [3]:
# 標準輸出 (stdout)
write(stdout, "Welcome to \nJulia!")

Welcome to 
Julia!

18

In [19]:
# 上面的例子中, out 的部分也輸出了成功輸出的字元數, 如果不要顯示的話, 可以加入 ; 隠藏
write(stdout, "Welcome to \nJulia!");

Welcome to 
Julia!

In [20]:
# 標準錯誤 (stderr)
write(stderr, "This is an error.")

This is an error.

17

### 1.2 檔案讀取

要操作文件檔案時，需要的時候可以透過函式 open() 及 close() 進行開啟和關閉。

開啟檔案時，預設的模式是唯讀。若需要進行寫入或添加 (append) 的話，需要指定相對應的模式。

|模式 (Mode)|操作說明|
|---|---|
|r|開啟並為唯讀。|
|r+|開啟可讀且可寫入。|
|w|檔案不存在時建立檔案、存在的話清空內容，可寫入。|
|w+|可讀且可寫入，檔案不存在時建立檔案、存在的話清空內容。|
|a|檔案不存在時建立檔案、存在的話添加內容，可寫入。|
|a+|可讀且可寫入，檔案不存在時建立檔案、存在的話添加內容。|

In [28]:
# 逐行讀取文字檔並顯示內容
f = open("iris.names")

while !eof(f)
    println(readline(f))
end

1. Title: Iris Plants Database
	Updated Sept 21 by C.Blake - Added discrepency information

2. Sources:
     (a) Creator: R.A. Fisher
     (b) Donor: Michael Marshall (MARSHALL%PLU@io.arc.nasa.gov)
     (c) Date: July, 1988

3. Past Usage:
   - Publications: too many to mention!!!  Here are a few.
   1. Fisher,R.A. "The use of multiple measurements in taxonomic problems"
      Annual Eugenics, 7, Part II, 179-188 (1936); also in "Contributions
      to Mathematical Statistics" (John Wiley, NY, 1950).
   2. Duda,R.O., & Hart,P.E. (1973) Pattern Classification and Scene Analysis.
      (Q327.D83) John Wiley & Sons.  ISBN 0-471-22361-1.  See page 218.
   3. Dasarathy, B.V. (1980) "Nosing Around the Neighborhood: A New System
      Structure and Classification Rule for Recognition in Partially Exposed
      Environments".  IEEE Transactions on Pattern Analysis and Machine
      Intelligence, Vol. PAMI-2, No. 1, 67-71.
      -- Results:
         -- very low misclassification rates (0% for t

透過下列函式查詢 IOStream 物件的屬性。

In [31]:
println("已開啟的檔案物件的類型:", typeof(f))

# 察看 IOStream 的狀態
println("是否已開啟: ", isopen(f))
println("是否可讀取: ", isreadable(f))
println("是否為唯讀: ", isreadonly(f)) # 預設未設定模式的話, 開啟後為唯讀
println("是否可寫入: ", iswritable(f))

已開啟的檔案物件的類型:IOStream
是否已開啟: true
是否可讀取: true
是否為唯讀: true
是否可寫入: false


In [32]:
close(f)

#### `readlines()` 函式

In [33]:
f = open("iris.names")
# 使用 readlines() 函式逐行讀取內容, 傳回值為向量類型
str_vec = readlines(f)
close(f)

typeof(str_vec)

Array{String,1}

### 1.3 檔案寫入

In [35]:
# 開啟檔案, 若檔案不存在就新建; 檔案存在的話, 寫入時不覆蓋原內容, 將新內容繼續添加至檔案中.
f = open("hello.txt", "a+")

println("已開啟的檔案物件的類型:", typeof(f))

# 察看 IOStream 的狀態
println("是否已開啟: ", isopen(f))
println("是否可讀取: ", isreadable(f))
println("是否為唯讀: ", isreadonly(f))
println("是否可寫入: ", iswritable(f))

write(f, "Hello\nJulia.")
write(f, "Learning Julia is fun!")

close(f)

已開啟的檔案物件的類型:IOStream
是否已開啟: true
是否可讀取: true
是否為唯讀: false
是否可寫入: true


### 1.4 讀寫緩衝 (IOBuffer)

有時候因為傳輸及處理的需求，我們需要先將 stream 裡面的資料暫存，再進行後續的處理，這時候就可以使用 `IOBuffer`。

`IOBuffer` 內容除了資料流本身之外，也包含了豐富的屬性來標示及做為操作 buffer 之用。使用 `dump()` 函式可以列出資料及所有屬性及其值。

透過 `take!()` 函式，可以取出 buffer 內容並重置 buffer。

In [39]:
# 初始化緩衝區, 設定最大大小為5個字元
buff = IOBuffer(maxsize = 5)

IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=5, ptr=1, mark=-1)

In [40]:
# 可以看出緩衝區的內容
buff.data

5-element Array{UInt8,1}:
 0x00
 0x00
 0x00
 0x00
 0x00

In [46]:
# 欲寫入6個字元到 buffer內, 但因超過緩衝區 maxsize, 所以僅成功寫入5個字元
write(buff, "123456")
buff.data

5-element Array{UInt8,1}:
 0x31
 0x32
 0x33
 0x34
 0x35

In [49]:
println("緩衝區大小: ", buff.size)
println("緩衝區目前游標位置 (為 size = 1): ", buff.ptr)

緩衝區大小: 5
緩衝區目前游標位置 (為 size = 1): 6


In [50]:
# 因為緩衝區已滿, 所以寫入成功字元數為 0
write(buff, "789")

0

In [52]:
# 查看緩衝區狀態
dump(buff)

Base.GenericIOBuffer{Array{UInt8,1}}
  data: Array{UInt8}((5,)) UInt8[0x31, 0x32, 0x33, 0x34, 0x35]
  readable: Bool true
  writable: Bool true
  seekable: Bool true
  append: Bool false
  size: Int64 5
  maxsize: Int64 5
  ptr: Int64 6
  mark: Int64 -1


In [55]:
# 取出緩衝區內容並且重置緩衝區
take!(buff)

0-element Array{UInt8,1}

In [56]:
dump(buff)
# 可以看出緩衝區已經被清空回到初始狀態

Base.GenericIOBuffer{Array{UInt8,1}}
  data: Array{UInt8}((5,)) UInt8[0x02, 0x00, 0x00, 0x00, 0x00]
  readable: Bool true
  writable: Bool true
  seekable: Bool true
  append: Bool false
  size: Int64 0
  maxsize: Int64 5
  ptr: Int64 1
  mark: Int64 -1


### 1.5 序列化 (Serialization)

序列化可以將資料在不同的類型與 stream 之間互相轉換，例如在資料傳輸之前先將字串 `serialize()`，在收到資料後進行反序列化 `deserialize()` 還原資料。

In [61]:
using Serialization

In [62]:
# 序列化
str = "Hello Julia"

io = IOBuffer()
s = Serializer(io)
println(typeof(s))
Serialization.writeheader(s)
for i in str
    serialize(s, i)
end

dump(io)

Serializer{Base.GenericIOBuffer{Array{UInt8,1}}}
Base.GenericIOBuffer{Array{UInt8,1}}
  data: Array{UInt8}((32,)) UInt8[0x37, 0x4a, 0x4c, 0x09, 0x04, 0x00, 0x00, 0x00, 0x0f, 0x48  …  0x0f, 0x75, 0x0f, 0x6c, 0x0f, 0x69, 0x0f, 0x61, 0x00, 0x00]
  readable: Bool true
  writable: Bool true
  seekable: Bool true
  append: Bool false
  size: Int64 30
  maxsize: Int64 9223372036854775807
  ptr: Int64 31
  mark: Int64 -1


In [74]:
# 反序列化
io = IOBuffer(take!(io))

while !eof(io)
    @show deserialize(io)
end

deserialize(io) = 'H'
deserialize(io) = 'e'
deserialize(io) = 'l'
deserialize(io) = 'l'
deserialize(io) = 'o'
deserialize(io) = ' '
deserialize(io) = 'J'
deserialize(io) = 'u'
deserialize(io) = 'l'
deserialize(io) = 'i'
deserialize(io) = 'a'


## 2. CSV 及 JSON 檔案讀取及寫入

### 2.1 CSV.jl: 讀取及寫入 CSV 檔

In [75]:
using Pkg

In [76]:
# 如果尚未安裝, 需先安裝套件
Pkg.add(PackageSpec(name="CSV", version="0.5.26"))

   Updating registry at `C:\Users\qwerz\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 % %==>                                      ]  4.6 %                                     ]  6.4 %>                                    ]  9.6 %=====>                                   ]  11.8 %]  13.8 %                                ]  19.0 %Fetching: [========>                                ]  19.9 %4 %25.6 %.7 % % [============>                            ]  29.8 %]  34.3 %]  39.0 % [=================>                       ]  41.0 %Fetching: [==================>                      ]  42.6 %===================>                     ]  46.0 %                   ]  50.1 %>                   ]  51.5 %55.1 %                ]  57.6 %Fetching: [=========================>               ]  61.5 %]  62.3 % %============================>            ]  68.8 % %.4 %]  75.0 %         ]  76.7 %================================>        ]  79.7 % [==================================>      ]  83.6 % %====================================>    

  Resolving package versions...
   Updating `C:\Users\qwerz\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\qwerz\.julia\environments\v1.4\Manifest.toml`
 [no changes]


In [77]:
# 目前安裝版本
Pkg.installed()["CSV"]

┌ Warning: Pkg.installed() is deprecated
└ @ Pkg D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.4\Pkg\src\Pkg.jl:531


v"0.5.26"

In [78]:
using CSV

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1260


以下的資料集為 UCI Machine Learning Repository 的 Auto MPG Data Set

如果沒有 header 的話, 可以設定 header 為 false. delim 可以指定分隔符號, 若未指定的話, 讀取時會自動判斷.

在有空值的資料列, 讀取時會顯示 warning, 可以忽略

In [79]:
df = CSV.read("auto-mpg.data", header=false, delim=",")

thread = 1 warning: parsed expected 9 columns, but didn't reach end of line on data row: 29. Ignoring any extra columns on this row
thread = 1 warning: parsed expected 9 columns, but didn't reach end of line on data row: 33. Ignoring any extra columns on this row
thread = 1 warning: parsed expected 9 columns, but didn't reach end of line on data row: 127. Ignoring any extra columns on this row
thread = 1 warning: parsed expected 9 columns, but didn't reach end of line on data row: 331. Ignoring any extra columns on this row
thread = 1 warning: parsed expected 9 columns, but didn't reach end of line on data row: 337. Ignoring any extra columns on this row
thread = 1 warning: parsed expected 9 columns, but didn't reach end of line on data row: 355. Ignoring any extra columns on this row
thread = 1 warning: parsed expected 9 columns, but didn't reach end of line on data row: 375. Ignoring any extra columns on this row


,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Float64,Int64⍰,Float64,String,Float64⍰,Float64⍰,Float64,Float64,String
1,18.0,8,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
2,15.0,8,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
3,18.0,8,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
4,16.0,8,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
5,17.0,8,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino
6,15.0,8,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500
7,14.0,8,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala
8,14.0,8,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii
9,14.0,8,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina


空值會自動取代為 missing 值

In [83]:
df[29, :]

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Float64,Int64⍰,Float64,String,Float64⍰,Float64⍰,Float64,Float64,String
29,9.0,missing,8.0,304.0,193.0,4732.0,18.5,70.0,1


如果沒有 header 的話, 也可以將 column 名稱設定至 header

In [84]:
df = CSV.read("auto-mpg.data", header=["mpg", "cylinders", "displacement", "horsepower", "weight", "acceleration", "model year", "origin", "car name"], delim=',')

thread = 1 warning: parsed expected 9 columns, but didn't reach end of line on data row: 29. Ignoring any extra columns on this row
thread = 1 warning: parsed expected 9 columns, but didn't reach end of line on data row: 33. Ignoring any extra columns on this row
thread = 1 warning: parsed expected 9 columns, but didn't reach end of line on data row: 127. Ignoring any extra columns on this row
thread = 1 warning: parsed expected 9 columns, but didn't reach end of line on data row: 331. Ignoring any extra columns on this row
thread = 1 warning: parsed expected 9 columns, but didn't reach end of line on data row: 337. Ignoring any extra columns on this row
thread = 1 warning: parsed expected 9 columns, but didn't reach end of line on data row: 355. Ignoring any extra columns on this row
thread = 1 warning: parsed expected 9 columns, but didn't reach end of line on data row: 375. Ignoring any extra columns on this row


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year
,Float64,Int64⍰,Float64,String,Float64⍰,Float64⍰,Float64
1,18.0,8,307.0,130.0,3504.0,12.0,70.0
2,15.0,8,350.0,165.0,3693.0,11.5,70.0
3,18.0,8,318.0,150.0,3436.0,11.0,70.0
4,16.0,8,304.0,150.0,3433.0,12.0,70.0
5,17.0,8,302.0,140.0,3449.0,10.5,70.0
6,15.0,8,429.0,198.0,4341.0,10.0,70.0
7,14.0,8,454.0,220.0,4354.0,9.0,70.0
8,14.0,8,440.0,215.0,4312.0,8.5,70.0
9,14.0,8,455.0,225.0,4425.0,10.0,70.0


In [85]:
# DataFrames 的用法, 會在之後的內容中進行詳細介紹和實作
# 此處僅是呼叫 DataFrames.show() 函式來顯示所有 column

# 如果尚未安裝, 需先安裝套件
Pkg.add(PackageSpec(name="DataFrames", version="0.20.2"))

using DataFrames

# 顥示所有 column
show(df[1:5, :], allcols=true)

  Resolving package versions...
   Updating `C:\Users\qwerz\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\qwerz\.julia\environments\v1.4\Manifest.toml`
 [no changes]


5×9 DataFrame
│ Row │ mpg     │ cylinders │ displacement │ horsepower │ weight   │
│     │ Float64 │ Int64⍰    │ Float64      │ String     │ Float64⍰ │
├─────┼─────────┼───────────┼──────────────┼────────────┼──────────┤
│ 1   │ 18.0    │ 8         │ 307.0        │ 130.0      │ 3504.0   │
│ 2   │ 15.0    │ 8         │ 350.0        │ 165.0      │ 3693.0   │
│ 3   │ 18.0    │ 8         │ 318.0        │ 150.0      │ 3436.0   │
│ 4   │ 16.0    │ 8         │ 304.0        │ 150.0      │ 3433.0   │
│ 5   │ 17.0    │ 8         │ 302.0        │ 140.0      │ 3449.0   │

│ Row │ acceleration │ model year │ origin  │ car name                  │
│     │ Float64⍰     │ Float64    │ Float64 │ String                    │
├─────┼──────────────┼────────────┼─────────┼───────────────────────────┤
│ 1   │ 12.0         │ 70.0       │ 1.0     │ chevrolet chevelle malibu │
│ 2   │ 11.5         │ 70.0       │ 1.0     │ buick skylark 320         │
│ 3   │ 11.0         │ 70.0       │ 1.0     │ plymouth satellit

#### 從 Zip 壓縮檔讀取 CSV 檔案

In [86]:
# 如果尚未安裝, 需先安裝套件
Pkg.add(PackageSpec(name="ZipFile", version="0.8.4"))

  Resolving package versions...
  Installed ZipFile ─ v0.8.4
   Updating `C:\Users\qwerz\.julia\environments\v1.4\Project.toml`
  [a5390f91] + ZipFile v0.8.4
   Updating `C:\Users\qwerz\.julia\environments\v1.4\Manifest.toml`
  [a5390f91] + ZipFile v0.8.4
   Building ZipFile → `C:\Users\qwerz\.julia\packages\ZipFile\DW0Qr\deps\build.log`


In [87]:
using ZipFile

z = ZipFile.Reader("iris.zip")
df = CSV.read(z.files[1], header=false)

┌ Info: Precompiling ZipFile [a5390f91-8eb1-5f08-bee0-b1d1ffed6cea]
└ @ Base loading.jl:1260


,Column1,Column2,Column3,Column4,Column5
,Float64,Float64,Float64,Float64,String
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
6,5.4,3.9,1.7,0.4,Iris-setosa
7,4.6,3.4,1.4,0.3,Iris-setosa
8,5.0,3.4,1.5,0.2,Iris-setosa
9,4.4,2.9,1.4,0.2,Iris-setosa


#### 將 DataFrame 寫入 CSV 檔案

In [88]:
CSV.write("a.csv", df, delim=";")

"a.csv"

### 2.2. JSON.jl: 讀取及寫入 JSON 檔

In [89]:
# 如果尚未安裝, 需先安裝套件
Pkg.add(PackageSpec(name="JSON", version="0.21.0"))

  Resolving package versions...
   Updating `C:\Users\qwerz\.julia\environments\v1.4\Project.toml`
  [682c06a0] + JSON v0.21.0
   Updating `C:\Users\qwerz\.julia\environments\v1.4\Manifest.toml`
 [no changes]


In [90]:
using JSON

從 JSON 檔讀取 UCI Machine Learning Repository Iris data set

In [91]:
j = JSON.parsefile("iris.json")

150-element Array{Any,1}:
 Dict{String,Any}("petalLength" => 1.4,"sepalWidth" => 3.5,"sepalLength" => 5.1,"petalWidth" => 0.2,"species" => "setosa")
 Dict{String,Any}("petalLength" => 1.4,"sepalWidth" => 3.0,"sepalLength" => 4.9,"petalWidth" => 0.2,"species" => "setosa")
 Dict{String,Any}("petalLength" => 1.3,"sepalWidth" => 3.2,"sepalLength" => 4.7,"petalWidth" => 0.2,"species" => "setosa")
 Dict{String,Any}("petalLength" => 1.5,"sepalWidth" => 3.1,"sepalLength" => 4.6,"petalWidth" => 0.2,"species" => "setosa")
 Dict{String,Any}("petalLength" => 1.4,"sepalWidth" => 3.6,"sepalLength" => 5.0,"petalWidth" => 0.2,"species" => "setosa")
 Dict{String,Any}("petalLength" => 1.7,"sepalWidth" => 3.9,"sepalLength" => 5.4,"petalWidth" => 0.4,"species" => "setosa")
 Dict{String,Any}("petalLength" => 1.4,"sepalWidth" => 3.4,"sepalLength" => 4.6,"petalWidth" => 0.3,"species" => "setosa")
 Dict{String,Any}("petalLength" => 1.5,"sepalWidth" => 3.4,"sepalLength" => 5.0,"petalWidth" => 0.2,"species" => 

In [92]:
typeof(j)

Array{Any,1}

#### 儲存 JSON 檔案

將物件轉換為 JSON 字串, 並存入 JSON 檔案

In [93]:
str = JSON.json(j)

"[{\"petalLength\":1.4,\"sepalWidth\":3.5,\"sepalLength\":5.1,\"petalWidth\":0.2,\"species\":\"setosa\"},{\"petalLength\":1.4,\"sepalWidth\":3.0,\"sepalLength\":4.9,\"petalWidth\":0.2,\"species\":\"setosa\"},{\"petalLength\":1.3,\"sepalWidth\":3.2,\"sepalLength\":4.7,\"petalWidth\":0.2,\"species\":\"setosa\"},{\"petalLength\":1.5,\"sepalWidth\":3.1,\"sepalLength\":4.6,\"petalWidth\":0.2,\"species\":\"setosa\"},{\"petalLength\":1.4,\"sepalWidth\":3.6,\"sepalLength\":5.0,\"petalWidth\":0.2,\"species\":\"setosa\"},{\"petalLength\":1.7,\"sepalWidth\":3.9,\"sepalLength\":5.4,\"petalWidth\":0.4,\"species\":\"setosa\"},{\"petalLength\":1.4,\"sepalWidth\":3.4,\"sepalLength\":4.6,\"petalWidth\":0.3,\"species\":\"setosa\"},{\"petalLength\":1.5,\"sepalWidth\":3.4,\"sepalLength\":5.0,\"petalWidth\":0.2,\"species\":\"setosa\"},{\"petalLength\":1.4,\"sepalWidth\":2.9,\"sepalLength\":4.4,\"petalWidth\":0.2,\"species\":\"setosa\"},{\"petalLength\":1.5,\"sepalWidth\":3.1,\"sepalLength\":4.9,\"petalWidt

In [95]:
f = open("b.json", "w")
println(f, str)
close(f)

## 3. 資料庫連線

![](https://avatars3.githubusercontent.com/u/6539129?s=200&v=4)

以下範例將以 JuliaDatabases 的套件示範。

GitHub: [https://github.com/JuliaDatabases](https://github.com/JuliaDatabases)

### 3.1 透過 ODBC.jl 連接 SQLite 資料庫

在 Windows 10 中要透過 ODBC 連接 SQLite 資料庫，須先安裝及設定 ODBC Driver for SQLite，範例中以 devart 的驅動程式為例。

免費下載試用版路徑為 [ODBC Driver for SQLite](https://www.devart.com/odbc/sqlite/download.html)。

設定 devart ODBC Driver for SQLite ，請參考文件：[Connecting to SQLite](https://www.devart.com/odbc/sqlite/docs/driver_configuration_and_conne.htm)

另外，需安裝 `ODBC.jl` 套件，在 Julia 中呼叫 ODBC 介面。

In [97]:
# 如果尚未安裝, 需先安裝套件
Pkg.add("ODBC")

  Resolving package versions...
  Installed CompilerSupportLibraries_jll ─ v0.3.3+0
  Installed OpenSpecFun_jll ────────────── v0.5.3+3
  Installed DecFP ──────────────────────── v0.4.10
  Installed SpecialFunctions ───────────── v0.9.0
  Installed ODBC ───────────────────────── v0.9.0
   Updating `C:\Users\qwerz\.julia\environments\v1.4\Project.toml`
  [be6f12e9] + ODBC v0.9.0
   Updating `C:\Users\qwerz\.julia\environments\v1.4\Manifest.toml`
  [e66e0078] + CompilerSupportLibraries_jll v0.3.3+0
  [55939f99] + DecFP v0.4.10
  [be6f12e9] + ODBC v0.9.0
  [efe28fd5] + OpenSpecFun_jll v0.5.3+3
  [276daf66] + SpecialFunctions v0.9.0
   Building DecFP → `C:\Users\qwerz\.julia\packages\DecFP\quvcm\deps\build.log`


In [98]:
using ODBC

┌ Info: Precompiling ODBC [be6f12e9-ca4f-5eb2-a339-a4f995cc0291]
└ @ Base loading.jl:1260


利用 DSN 連接資料來源, 在 Windows 環境中, 須先到控制台的系統管理工具中, 設定 ODBC 資料來源, 下面範例的 DSN 名稱為 "iris-dsn", 資料庫中 table 名稱為 "iris".

設定 DSN 時, 請將資料庫檔案為 iris_dataset.db

In [99]:
dsn = ODBC.DSN("iris_dsn")
df = ODBC.query(dsn, "select * from iris")

ODBC.ODBCError: ODBC.ODBCError("API.SQLExecDirect(stmt, query) failed; return code: -2 => SQL_INVALID_HANDLE")

In [100]:
ODBC.disconnect!(dsn)

### 3.2 透過原生套件連接 SQLite 資料庫

In [101]:
# 如果尚未安裝, 需先安裝套件
Pkg.add(PackageSpec(name="SQLite", version="0.8.1"))

  Resolving package versions...
  Installed SQLite ─ v0.8.1
   Updating `C:\Users\qwerz\.julia\environments\v1.4\Project.toml`
  [0aa819cd] + SQLite v0.8.1
   Updating `C:\Users\qwerz\.julia\environments\v1.4\Manifest.toml`
  [0aa819cd] + SQLite v0.8.1
   Building SQLite → `C:\Users\qwerz\.julia\packages\SQLite\yKARA\deps\build.log`


In [102]:
# 目前安裝版本
Pkg.installed()["SQLite"]

┌ Warning: Pkg.installed() is deprecated
└ @ Pkg D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.4\Pkg\src\Pkg.jl:531


v"0.8.1"

In [103]:
using SQLite

┌ Info: Precompiling SQLite [0aa819cd-b072-5ff4-a722-6bc24af294d9]
└ @ Base loading.jl:1260


In [104]:
# 從 SQLite 資料庫中讀取資料並傳回 DataFrame
cond = "Iris-setosa"

db = SQLite.DB("iris_dataset.db")
df = SQLite.Query(db, "SELECT * FROM iris where species = \'$cond\'") |> DataFrame

,sepal_length,sepal_width,petal_length,petal_width,species
,Float64⍰,Float64⍰,Float64⍰,Float64⍰,String⍰
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
6,5.4,3.9,1.7,0.4,Iris-setosa
7,4.6,3.4,1.4,0.3,Iris-setosa
8,5.0,3.4,1.5,0.2,Iris-setosa
9,4.4,2.9,1.4,0.2,Iris-setosa
